In [19]:
## document data structure
from langchain_core.documents import Document

In [5]:
doc=Document(
    page_content="Hello, world!",
     metadata={"source": "example.com"}
     )

In [18]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "../data/docker.pdf"
loader = PyMuPDFLoader(file_path)
docs = loader.load()

In [20]:
docs

[Document(metadata={'producer': 'XeTeX 0.99998', 'creator': 'LaTeX with hyperref package', 'creationdate': '2025-05-12T21:28:19+00:00', 'source': '../data/docker.pdf', 'file_path': '../data/docker.pdf', 'total_pages': 280, 'format': 'PDF 1.5', 'title': 'Docker Deep Dive', 'author': 'Nigel Poulton', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250512212819-00'00'", 'page': 0}, page_content=''),
 Document(metadata={'producer': 'XeTeX 0.99998', 'creator': 'LaTeX with hyperref package', 'creationdate': '2025-05-12T21:28:19+00:00', 'source': '../data/docker.pdf', 'file_path': '../data/docker.pdf', 'total_pages': 280, 'format': 'PDF 1.5', 'title': 'Docker Deep Dive', 'author': 'Nigel Poulton', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250512212819-00'00'", 'page': 1}, page_content='Docker Deep Dive\nZero to Docker in a single book!\nNigel Poulton\nThis book is available at https://leanp

In [21]:
len(docs)

280

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,separators=["\n\n", "\n", " ", ""])
texts = text_splitter.split_documents(docs)

In [23]:
print(len(texts))
texts

574


[Document(metadata={'producer': 'XeTeX 0.99998', 'creator': 'LaTeX with hyperref package', 'creationdate': '2025-05-12T21:28:19+00:00', 'source': '../data/docker.pdf', 'file_path': '../data/docker.pdf', 'total_pages': 280, 'format': 'PDF 1.5', 'title': 'Docker Deep Dive', 'author': 'Nigel Poulton', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250512212819-00'00'", 'page': 1}, page_content='Docker Deep Dive\nZero to Docker in a single book!\nNigel Poulton\nThis book is available at https://leanpub.com/dockerdeepdive\nThis version was published on 2025-05-12\nISBN 9781916585133\nThis is a Leanpub book. Leanpub empowers authors and publishers with the Lean\nPublishing process. Lean Publishing is the act of publishing an in-progress ebook using\nlightweight tools and many iterations to get reader feedback, pivot until you have the\nright book and build traction once you do.\n© 2016 - 2025 Nigel Poulton'),
 Document(metadata={'producer': '

In [24]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [25]:
len(texts)

574

In [26]:
texts

[Document(metadata={'producer': 'XeTeX 0.99998', 'creator': 'LaTeX with hyperref package', 'creationdate': '2025-05-12T21:28:19+00:00', 'source': '../data/docker.pdf', 'file_path': '../data/docker.pdf', 'total_pages': 280, 'format': 'PDF 1.5', 'title': 'Docker Deep Dive', 'author': 'Nigel Poulton', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250512212819-00'00'", 'page': 1}, page_content='Docker Deep Dive\nZero to Docker in a single book!\nNigel Poulton\nThis book is available at https://leanpub.com/dockerdeepdive\nThis version was published on 2025-05-12\nISBN 9781916585133\nThis is a Leanpub book. Leanpub empowers authors and publishers with the Lean\nPublishing process. Lean Publishing is the act of publishing an in-progress ebook using\nlightweight tools and many iterations to get reader feedback, pivot until you have the\nright book and build traction once you do.\n© 2016 - 2025 Nigel Poulton'),
 Document(metadata={'producer': '

In [28]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")




In [29]:

from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(texts))]
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=os.getenv("pinecone_api_key"))
index_name = "rag-01"
pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )

index = pc.Index(index_name)

  





In [30]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index = index, embedding=embeddings)
vector_store.add_documents(documents=texts, ids=uuids)

['21712b2b-658b-4c69-9431-409bb09d2447',
 '39bdc125-602a-496a-b674-dfa254effcee',
 '388fc58d-aacf-418b-bcc9-07d057070c02',
 '83a3f19c-b186-4d96-a6b5-72096127aca5',
 '1ee438dd-a223-4e59-9d2e-b9e5f2f8de66',
 '1a3a6624-e5d0-4879-bf46-fa02c216118c',
 'dae615a5-a613-4566-ba95-4b7e3782e7a1',
 '5cabcf02-3207-4f71-a32c-7e638855ab0b',
 '7049224d-200e-49aa-a345-a81336d301a5',
 '14584d99-a4ba-4a6b-a0bc-9d0e7cf55e81',
 '5420c922-0ef6-4052-90db-2efd35a5456a',
 '8d0ee4b0-a3fd-4e7d-b8db-58213321b6a6',
 '88d2a08f-1fb9-46a7-8cb4-0ccf3229d4fe',
 'a4d321d7-59fd-485f-b993-fb406b8144c3',
 'ae6e8748-9619-4d19-8c86-ce6f17023726',
 '9883fc31-38c0-44ef-b961-7ec431a46b2c',
 '57d7f2a2-9d2e-4c83-9e97-496d90bb3eaa',
 'bd1262c5-ce00-47c4-8090-ee17321e8e45',
 'a3cd1615-f4d1-47cc-b8c4-f0b1b1af4ec8',
 'cd147d1b-9fe3-4334-8b43-8881fbde310d',
 '42c76f9e-0593-4552-8c03-ef2d6847543f',
 '75b0b029-9211-4d40-ae2a-dd1f7de21bf0',
 '98c81913-cf0f-4417-bac2-e64ec1c93749',
 '7036223f-005f-4d4e-a05e-bf74f6d90bb4',
 'e8eb6960-536c-

In [31]:
from langchain_core.messages import AIMessage


In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(api_key=os.getenv("GOOGLE_API_KEY"), model="gemini-2.0-flash")

In [34]:
user_query = input("How may I help you")

retrieved_docs = vector_store.similarity_search(user_query, k=3)
context = [doc.page_content for doc in retrieved_docs]


template = f"""You are an LLM which answers queries regarding Docker ,
            you have access to a RAG knowlege source which is a complete docker deep dive book
            Here is the user query {user_query}. Answer with respect to the retrieved documents from the RAG
            pipeline using similarity/cosine search . Retrieved Docs : {context}
            """

prompt = template.format(user_query=user_query, context=context)
ans = llm.invoke(prompt)
ans.content



'To run a Docker image, you need to start a container from that image. The provided documents mention using commands like `docker version` to ensure Docker is installed and running, and they illustrate running a container based on the `nginx:latest` image in detached mode using the `-p` flag to map ports.  However, the exact command to run a docker image is not explicitly stated in the given extracts, but it is implied that you use `docker run <image_name>`'